# Analyse des retards aériens

## Structure du dataset

| Variable              | Description courte                                                                    | Type    | Exemples                     |
| --------------------- | ------------------------------------------------------------------------------------- | ------- | ---------------------------- |
| `year`                | Année de collecte des données                                                         | int     | 2023, 2024                   |
| `month`               | Mois (représenté numériquement)                                                       | int     | 1, 6, 12                     |
| `carrier`             | Code du transporteur aérien                                                           | object  | 'AA', 'DL', 'UA'             |
| `carrier_name`        | Nom complet du transporteur aérien                                                    | object  | 'American Airlines', 'Delta' |
| `airport`             | Code de l'aéroport (IATA)                                                             | object  | 'JFK', 'LAX', 'ATL'          |
| `airport_name`        | Nom complet de l'aéroport                                                             | object  | 'John F. Kennedy Intl'       |
| `arr_flights`         | Nombre de vols arrivant à l'aéroport                                                  | int     | 1500, 2300                   |
| `arr_del15`           | Nombre de vols en retard de plus de 15 minutes                                        | int     | 320, 450                     |
| `carrier_ct`          | Nombre de vols retardés à cause de la compagnie aérienne (ex : équipage indisponible) | float   | 45, 60                       |
| `weather_ct`          | Nombre de vols retardés en raison de la météo                                         | float   | 10, 25                       |
| `nas_ct`              | Nombre de vols retardés par le système national de l'aviation (ex : trafic aérien)    | float   | 100, 120                     |
| `security_ct`         | Nombre de vols annulés pour des raisons de sécurité                                   | float   | 0, 2                         |
| `late_aircraft_ct`    | Nombre de vols retardés à cause de l'arrivée tardive d'un avion précédent             | float   | 80, 95                       |
| `arr_cancelled`       | Nombre de vols annulés                                                                | int     | 15, 20                       |
| `arr_diverted`        | Nombre de vols déroutés vers un autre aéroport                                        | int     | 3, 5                         |
| `arr_delay`           | Temps total (en minutes) de retard des vols                                           | float   | 1240.5, 2130.0               |
| `carrier_delay`       | Durée totale (en minutes) de retard causé par la compagnie aérienne                   | float   | 300.0, 450.5                 |
| `weather_delay`       | Durée totale (en minutes) de retard causé par la météo                                | float   | 120.0, 80.0                  |
| `nas_delay`           | Durée totale (en minutes) de retard causé par le système de navigation aérienne       | float   | 500.0, 620.0                 |
| `security_delay`      | Durée totale (en minutes) de retard causé par un problème de sécurité                 | float   | 0.0, 5.0                     |
| `late_aircraft_delay` | Durée totale (en minutes) de retard causé par un vol précédent en retard              | float   | 320.0, 470.0                 |

## Import des dépendances et du dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Chemin du dataset
csv_path = '../data/extracted/USA_Airline_Delay_Cause/Airline_Delay_Cause.csv'

## Vérification

In [ ]:
# Chargement des donnnées
df = pd.read_csv(csv_path)

# Aperçu de la structure
print("Shape (lignes, colonnes) :", df.shape)
print("\nColonnes :", df.columns.tolist())
display(df.head(3))

## Nettoyage des données

In [ ]:
## Typages des données

# Affichage des types de données
print("Types de données :")
print(df.dtypes)

## Fusions des colonnes

# Fusion des colonnes 'year' et 'month' en une colonne 'date'
df['date'] = df['year'].astype(str) + '-' + df['month'].astype(str).str.zfill(2)

# Vérification
print("\nDate après conversion :")
print(df[['date']].head(5))

## Traitement des valeurs manquantes

print("\nNombre de valeurs manquantes par colonne :")
print(df.isnull().sum())

# Suppression des lignes sans vols
df = df.dropna(subset=['arr_flights'])

# Remplissage de arr_del15 avec 0 si vide
df['arr_del15'] = df['arr_del15'].fillna(0)

# Vérification
print("\nValeurs manquantes après traitement :")
print(df.isnull().sum())

## Statistiques

In [ ]:
# Aperçu des stats globales
print("\nRésumé statistique rapide :")
display(df.describe().T.round(2))

### Calcul du score de risque et de résilience

Ce bloc calcule plusieurs indicateurs pour chaque ligne du dataset :

- `risk_score` : proportion de vols perturbés (retard >15 min, annulés ou déroutés) par rapport au nombre total de vols.
- `mttr` (Mean Time To Recovery) : temps moyen de retard par incident, toutes causes confondues.
- `mttr_norm` : version normalisée du MTTR pour faciliter la comparaison.
- `delay_rate` : taux de vols en retard (>15 min).
- `resilience_score` : indicateur de résilience, plus il est élevé, plus l'aéroport ou la compagnie gère efficacement les perturbations.

Ces indicateurs permettent d’identifier les aéroports ou compagnies les plus exposés aux risques de retard et ceux qui récupèrent le plus rapidement.

In [ ]:
# Calcul du score de risque
df['risk_score'] = (df['arr_del15'] + df['arr_cancelled'] + df['arr_diverted']) / df['arr_flights']

# MTTR global
df['mttr'] = df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].sum(axis=1) / \
             df[['carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct']].sum(axis=1)

# Normalisation du MTTR
df['mttr_norm'] = (df['mttr'] - df['mttr'].min()) / (df['mttr'].max() - df['mttr'].min())

# Taux de retard
df['delay_rate'] = df['arr_del15'] / df['arr_flights']

# Score de résilience
df['resilience_score'] = 1 - (df['mttr_norm'] + df['delay_rate'])

# Vérification
print("\nAperçu des nouvelles colonnes :")
print(df[['risk_score', 'mttr', 'mttr_norm', 'delay_rate', 'resilience_score']].head(5))

### Vols retardé par cause

In [ ]:
# Agrégation des causes de retard
causes = df[['carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct']].sum()

# Afficher le nombre de vols total
print("\nNombre total de vols : {:,}".format(df['arr_flights'].sum()).replace(",", " "))
print("Nombre de vols retardés : {:,}".format(df['arr_del15'].sum()).replace(",", " "))

# Graphique en barres
causes.plot(kind='bar', figsize=(8,5), title='Nombre de vols retardés par cause', ylabel='Nombre de vols')
plt.grid(True)
plt.tight_layout()

### Ratio de retard par années

In [ ]:
# Grouper par année et calculer le ratio de retard
delay_ratio_per_year = (df.groupby('year')['arr_del15'].sum() / df.groupby('year')['arr_flights'].sum())

# Affichage
delay_ratio_per_year.plot(kind='bar', title='Ratio de retard (>15 min) par année', ylabel='Ratio de retard')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

### Durée totale des retards par cause

In [ ]:
# Agrégation des durées de retard
delays = df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].sum()

# Graphique
delays.plot(kind='bar', color='orange', figsize=(8,5), title='Durée totale des retards par cause', ylabel='Minutes de retard')
plt.grid(True)
plt.tight_layout()

## Export du dataset préparé pour le dashboard

In [ ]:
df.to_csv("../data/cleaned/airline_delay_cause_cleaned.csv", index=False)
print("Fichier enregistré : /data/cleaned/airline_delay_cause_cleaned.csv")